#מגישים
###שניר פרימוביץ 211518139
###יוסף סרור 209452945
###איתי אמוס 313348104
###דותן חזוט 315779926

#חלק ראשון: הכנה והכרות עם הדאטה

##המרת הקובץ לפורמט csv

In [ ]:
import pandas as pd

# Load the TSV file
tsv_file_path = '/content/drive/MyDrive/Restaurant_Reviews.tsv'
df = pd.read_csv(tsv_file_path, sep='\t')

# Save as CSV
csv_file_path = '/content/drive/MyDrive/Restaurant_Reviews.csv'
df.to_csv(csv_file_path, index=False)


'/content/drive/MyDrive/Restaurant_Reviews.csv'

##קוד שמוצא כמה ביקורות חיוביות יש וכמה שליליות

In [ ]:
# Count positive and negative reviews based on the 'Liked' column
positive_count = (df['Liked'] == 1).sum()
negative_count = (df['Liked'] == 0).sum()

print("Number of positive reviews:", positive_count)
print("Number of negative reviews:", negative_count)


Number of positive reviews: 500
Number of negative reviews: 500


##קוד אשר מוצא את האורך הממוצע לביקורת חיובית ולביקורת שלילית

In [ ]:
# Compute the length of each review
df['Review_Length'] = df['Review'].apply(len)

# Compute average length for positive and negative reviews
avg_length_positive = df[df['Liked'] == 1]['Review_Length'].mean()
avg_length_negative = df[df['Liked'] == 0]['Review_Length'].mean()

print("Average length of positive reviews(in characters):", avg_length_positive)
print("Average length of negative reviews(in characters):", avg_length_negative)


Average length of positive reviews(in characters): 55.898
Average length of negative reviews(in characters): 60.75


##ניתן להסיק מן הסעיפים הקודמים שקובץ המידע טוב מאחר והוא מכיל כמות מכובדת של ביקורות בעלי הרבה מלל אז יש הרבה ממה ללמוד והביקורות מגוונות

##מציאת 5 המילים הנפוצות ביותר בבביקורות השליליות

In [ ]:
import spacy
from collections import Counter

# Load the small English model from spaCy
nlp = spacy.load("en_core_web_sm")

# Filter only negative reviews
negative_reviews = df[df['Liked'] == 0]['Review']

# Process all negative reviews into one large text
all_negative_text = " ".join(negative_reviews)

# Use spaCy to process the text
doc = nlp(all_negative_text)

# Extract meaningful words (nouns, adjectives, verbs) excluding stop words and punctuation
tokens = [token.lemma_.lower() for token in doc
          if token.is_alpha and not token.is_stop and token.pos_ in ["NOUN", "ADJ", "VERB"]]

# Count the most common words
common_words = Counter(tokens).most_common(5)
print("The most common words in negative reviews are:")
for word, count in common_words:
    print(f"{word}: {count}")

The most common words in negative reviews are:
food: 67
place: 52
service: 38
bad: 32
time: 29


###מה ניתן להסיק מהמילים

1.   food - ביקורות על איכות האוכל
2.   place - ביקורות על המקום
3.   service - ביקורות על שירות
4.   bad - חוות דעת
5.   time - ביקורות כנראה על זמן ההגשה



##מציאת הביקורת הארוכה ביותר

In [ ]:
# Find the row with the maximum review length
longest_review_row = df.loc[df['Review_Length'].idxmax()]

longest_review_text = longest_review_row['Review']
longest_review_label = 'Positive' if longest_review_row['Liked'] == 1 else 'Negative'

print("The longest review is:")
print(longest_review_text)
print("Label:", longest_review_label)


The longest review is:
The problem I have is that they charge $11.99 for a sandwich that is no bigger than a Subway sub (which offers better and more amount of vegetables).
Label: Negative


##מציאת NER וסוגי הישויות

In [ ]:
# Process the text with spaCy
doc = nlp(longest_review_text)

# Extract and display named entities and their labels
for ent in doc.ents:
    print(f"Entity: '{ent.text}', Type: {ent.label_}")


Entity: '11.99', Type: MONEY
Entity: 'Subway', Type: ORG


#חלק שני : ייצוג וניתוח ביקורות על בסיס ישויות מזוהות (NER)

##שלב א: זיהוי ישויות בכל ביקורת

In [ ]:
import pandas as pd
import spacy

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Load your reviews file (adjust path if needed)
df = pd.read_csv('/content/drive/MyDrive/Restaurant_Reviews.csv')

# Function to extract entities from a single review
def extract_entities(text):
    doc = nlp(text)
    # Return list of (entity text, entity label)
    return [(ent.text, ent.label_) for ent in doc.ents]

# Apply to each review
df['Entities'] = df['Review'].apply(extract_entities)

# Optionally save to CSV for later analysis
df.to_csv("Reviews_with_Entities.csv", index=False)

# Display a sample
print(df[['Review', 'Entities']].head())


                                              Review  \
0                           Wow... Loved this place.   
1                                 Crust is not good.   
2          Not tasty and the texture was just nasty.   
3  Stopped by during the late May bank holiday of...   
4  The selection on the menu was great and so wer...   

                                            Entities  
0                                                 []  
1                                     [(Crust, ORG)]  
2                                                 []  
3  [(the late May bank, DATE), (Rick Steve, PERSON)]  
4                                                 []  


##שלב ב: יצירת ייצוג המבוסס על ישויות

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
from tqdm import tqdm

# Load spaCy
nlp = spacy.load("en_core_web_sm")

# Load the reviews data
df = pd.read_csv("/content/drive/MyDrive/Restaurant_Reviews.csv")

# Progress bar for entity extraction
tqdm.pandas()

# Extract entities as text
def extract_entity_text(text):
    doc = nlp(text)
    return " ".join(ent.text for ent in doc.ents)

df['Entity_Text'] = df['Review'].progress_apply(extract_entity_text)

# Build TF-IDF on entity representation
vectorizer = TfidfVectorizer()
X_entity = vectorizer.fit_transform(df['Entity_Text'])
y = df['Liked']

100%|██████████| 1000/1000 [00:09<00:00, 101.96it/s]


##שלב ג: אימון מודלים על בסיס ייצוג הישויות

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Split into Train/Test
X_train, X_test, y_train, y_test = train_test_split(
    X_entity, y, test_size=0.2, random_state=42
)

# Prepare models
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(n_estimators=100),
    "SVM": SVC(),
    "Multinomial NB": MultinomialNB(),
    "KNN": KNeighborsClassifier(n_neighbors=5)
}

# Train each model
for name, model in models.items():
    model.fit(X_train, y_train)

##שלב ד: הערכת ביצועים

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Evaluate each model
for name, model in models.items():
    print(f"\n==== {name} ====")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    print(f"Accuracy : {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall   : {rec:.4f}")
    print(f"F1 Score : {f1:.4f}")
    print("\nConfusion Matrix:")
    print(pd.DataFrame(cm,
                       index=['Actual Negative','Actual Positive'],
                       columns=['Predicted Negative','Predicted Positive']))


==== Logistic Regression ====
Accuracy : 0.4900
Precision: 0.5625
Recall   : 0.0865
F1 Score : 0.1500

Confusion Matrix:
                 Predicted Negative  Predicted Positive
Actual Negative                  89                   7
Actual Positive                  95                   9

==== Random Forest ====
Accuracy : 0.4950
Precision: 0.6000
Recall   : 0.0865
F1 Score : 0.1513

Confusion Matrix:
                 Predicted Negative  Predicted Positive
Actual Negative                  90                   6
Actual Positive                  95                   9

==== SVM ====
Accuracy : 0.4950
Precision: 0.6364
Recall   : 0.0673
F1 Score : 0.1217

Confusion Matrix:
                 Predicted Negative  Predicted Positive
Actual Negative                  92                   4
Actual Positive                  97                   7

==== Multinomial NB ====
Accuracy : 0.4900
Precision: 0.5625
Recall   : 0.0865
F1 Score : 0.1500

Confusion Matrix:
                 Predicted Negative

##שלב ה: ניתוח ראשוני

All your models ended up with:

~49-50% accuracy, barely above random guessing (which for a balanced dataset would be ~50%),

extremely low recall (~7-8%) and low F1 (~0.12-0.15).

This means:

The models almost always predict "Negative" (the dominant class in the confusion matrix rows), missing most positives.

Very few positive reviews are correctly identified.

👉 This tells us that relying only on named entities (extracted by NER) misses most of the semantic signal needed to classify reviews correctly.


---


Most of the sentiment in restaurant reviews is expressed through these opinion words, not the entities.
So a model only trained on entities loses almost all sentiment-bearing features.



---
Even though the performance is low, some types of entities could hint at positive experiences:

Locations: A mention of popular or upscale areas might be weakly associated with positive experiences.

E.g., “dinner at Times Square” might correlate with celebration or special occasions.

Organization / Restaurant Names: Known high-end places.

MONEY amounts: In some contexts, a review mentioning a higher spend might correlate with a special occasion (though not always positive).

EVENTS: Like “anniversary”, “birthday”.


#חלק שלישי : בניית מודלים על בסיס Bag of Words

##שלב א: הסרת סימני פיסוק

In [ ]:
import pandas as pd
import string

# Load the reviews data
df = pd.read_csv('/content/drive/MyDrive/Restaurant_Reviews.csv')

# Function to remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

# Apply to each review
df['Clean_Review'] = df['Review'].apply(remove_punctuation)

# Show before & after
print(df[['Review', 'Clean_Review']].head())


                                              Review  \
0                           Wow... Loved this place.   
1                                 Crust is not good.   
2          Not tasty and the texture was just nasty.   
3  Stopped by during the late May bank holiday of...   
4  The selection on the menu was great and so wer...   

                                        Clean_Review  
0                               Wow Loved this place  
1                                  Crust is not good  
2           Not tasty and the texture was just nasty  
3  Stopped by during the late May bank holiday of...  
4  The selection on the menu was great and so wer...  


### Why remove punctuation?
When building Bag of Words (or TF-IDF) models, we tokenize text into words.
Punctuation is usually noise for this because:

It doesn’t carry meaning for classification tasks like sentiment (punctuation isn’t usually tied to positive/negative opinions).

“good,” and “good” would be treated as different tokens if we keep the comma. This unnecessarily increases the feature space and splits frequency counts.

By removing punctuation, we standardize the tokens, reducing sparsity and improving the model’s ability to recognize repeated words.

##שלב ב: החזרת מילים לצורתן הבסיסית

In [ ]:
import pandas as pd
import spacy

# Lemmatize
def lemmatize_text(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])

df['Lemmatized_Review'] = df['Clean_Review'].progress_apply(lemmatize_text)

# Save to new CSV
df.to_csv("Restaurant_Reviews_Lemmatized.csv", index=False)

# Preview
print(df[['Clean_Review', 'Lemmatized_Review']].head())


100%|██████████| 1000/1000 [00:09<00:00, 106.35it/s]


                                        Clean_Review  \
0                               Wow Loved this place   
1                                  Crust is not good   
2           Not tasty and the texture was just nasty   
3  Stopped by during the late May bank holiday of...   
4  The selection on the menu was great and so wer...   

                                   Lemmatized_Review  
0                                wow love this place  
1                                  Crust be not good  
2            not tasty and the texture be just nasty  
3  stop by during the late May bank holiday off R...  
4  the selection on the menu be great and so be t...  


###What does “return to basic form” mean?
When building a text model, we want:

“runs”, “running”, “ran”
to all reduce to “run”.

This helps group related words under a single feature, so the model learns that they refer to the same concept.

###What did we choose and why?
We’ll use: lemmatization
Because it ensures valid dictionary words and handles context (verbs vs. nouns) better.

It’s slower, but for understanding restaurant reviews (where opinion words matter), accuracy is worth it.

##שלב ג: יצירת ייצוג בשיטת Bag of Words

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# Load the lemmatized data
df = pd.read_csv("Restaurant_Reviews_Lemmatized.csv")

# Build the Bag of Words
vectorizer = CountVectorizer()
X_bow = vectorizer.fit_transform(df['Lemmatized_Review'])

# Convert to DataFrame
bow_df = pd.DataFrame(X_bow.toarray(), columns=vectorizer.get_feature_names_out())

# Save to CSV
bow_df.to_csv("Bag_of_Words_Representation.csv", index=False)

# Show shape and first few rows
print("Shape of BoW matrix:", X_bow.shape)
print(bow_df.head())


Shape of BoW matrix: (1000, 1755)
   10  100  1199  12  15  17  1979  20  2007  23  ...  yelpers  yet  you  \
0   0    0     0   0   0   0     0   0     0   0  ...        0    0    0   
1   0    0     0   0   0   0     0   0     0   0  ...        0    0    0   
2   0    0     0   0   0   0     0   0     0   0  ...        0    0    0   
3   0    0     0   0   0   0     0   0     0   0  ...        0    0    0   
4   0    0     0   0   0   0     0   0     0   0  ...        0    0    0   

   your  yourself  yucky  yukon  yum  yummy  zero  
0     0         0      0      0    0      0     0  
1     0         0      0      0    0      0     0  
2     0         0      0      0    0      0     0  
3     0         0      0      0    0      0     0  
4     0         0      0      0    0      0     0  

[5 rows x 1755 columns]


### What is the Bag of Words method?
Bag of Words is a technique to represent text data numerically.

It converts a document (sentence, review, etc.) into a vector that counts how many times each word appears.

It ignores grammar, word order, punctuation, and focuses only on the frequency of words.

###Why is Bag of Words important?
It translates text into numbers so that machine learning models can use it.

Helps capture which words are most frequently used in reviews.

Often works surprisingly well for sentiment tasks, since words like "bad", "great", "dirty", "amazing" directly correlate with labels.

##שלב ד: אימון מודלים

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Load the lemmatized data
df = pd.read_csv("Restaurant_Reviews_Lemmatized.csv")

# Build Bag of Words
vectorizer = CountVectorizer()
X_bow = vectorizer.fit_transform(df['Lemmatized_Review'])
y = df['Liked']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X_bow, y, test_size=0.2, random_state=42
)

# Define models
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(n_estimators=100),
    "SVM": SVC(),
    "Multinomial NB": MultinomialNB(),
    "KNN": KNeighborsClassifier(n_neighbors=5)
}

# Train
for name, model in models.items():
    model.fit(X_train, y_train)

##שלב ה: הערכת ביצועי המודלים

In [ ]:
# predict and evaluate
for name, model in models.items():
    print(f"\n==== {name} ====")
    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    print(f"Accuracy : {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall   : {rec:.4f}")
    print(f"F1 Score : {f1:.4f}")
    print("\nConfusion Matrix:")
    print(pd.DataFrame(cm,
                       index=['Actual Negative', 'Actual Positive'],
                       columns=['Predicted Negative', 'Predicted Positive']))


==== Logistic Regression ====
Accuracy : 0.7600
Precision: 0.8111
Recall   : 0.7019
F1 Score : 0.7526

Confusion Matrix:
                 Predicted Negative  Predicted Positive
Actual Negative                  79                  17
Actual Positive                  31                  73

==== Random Forest ====
Accuracy : 0.7750
Precision: 0.8041
Recall   : 0.7500
F1 Score : 0.7761

Confusion Matrix:
                 Predicted Negative  Predicted Positive
Actual Negative                  77                  19
Actual Positive                  26                  78

==== SVM ====
Accuracy : 0.7750
Precision: 0.8105
Recall   : 0.7404
F1 Score : 0.7739

Confusion Matrix:
                 Predicted Negative  Predicted Positive
Actual Negative                  78                  18
Actual Positive                  27                  77

==== Multinomial NB ====
Accuracy : 0.7650
Precision: 0.8132
Recall   : 0.7115
F1 Score : 0.7590

Confusion Matrix:
                 Predicted Negative

##שלב ו: בחירת מודל והסקת מסקנות

###Analysis of model performance
####Overall
Random Forest and SVM show the best balanced performance:

*   Both have highest accuracy (0.775) and similar F1 scores (around 0.77).

Random Forest slightly edges out SVM on F1, which combines precision + recall.

Logistic Regression and Naive Bayes also performed well, but with slightly lower F1.

KNN lags clearly behind in accuracy (0.66) and precision (0.65).

####Looking deeper at confusion matrices
Random Forest and SVM had fewer false positives & false negatives, showing balanced sensitivity (recall) and correctness (precision).

KNN had a very high number of misclassifications on negatives (40).

###Model selection: Random Forest Classifier
####Why?
It achieved the highest F1 Score (0.7761), meaning it best balances precision & recall — which is crucial in sentiment analysis where both false positives and false negatives matter.

It also had the highest recall (0.7500) among the top performers, ensuring it catches more positive reviews correctly, while maintaining strong precision.

Random Forest is also robust to overfitting, handles noisy data well, and is relatively interpretable